https://data.seattle.gov/Public-Safety/Terry-Stops/28ny-9ts8

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import time
import os, sys

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', 100)

RANDOM_STATE = 100

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
path = os.path.join('..', 'data', 'raw', 'terry-stops.csv')
df = pd.read_csv(path, dtype='str')

# Strip spaces, as relevant
for col in df.columns:
    df[col] = df[col].str.strip()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47213 entries, 0 to 47212
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Subject Age Group         47213 non-null  object
 1   Subject ID                47213 non-null  object
 2   GO / SC Num               47213 non-null  object
 3   Terry Stop ID             47213 non-null  object
 4   Stop Resolution           47213 non-null  object
 5   Weapon Type               47213 non-null  object
 6   Officer ID                47213 non-null  object
 7   Officer YOB               47213 non-null  object
 8   Officer Gender            47213 non-null  object
 9   Officer Race              47213 non-null  object
 10  Subject Perceived Race    47213 non-null  object
 11  Subject Perceived Gender  47213 non-null  object
 12  Reported Date             47213 non-null  object
 13  Reported Time             47213 non-null  object
 14  Initial Call Type     

In [14]:
# ------ CLEAN DATA ONE COLUMN AT A TIME -------

# Subject Age Group
df['Subject Age Group'].replace('-', np.nan, inplace=True)

# Subject ID
df['Subject ID'].replace('-1', np.nan, inplace=True)

# Weapon Type
df['Weapon Type'].replace('-', 'None', inplace=True)
df['Weapon Type'].replace('Firearm Other', 'Firearm', inplace=True)
df['Weapon Type'].replace('Other Firearm', 'Firearm', inplace=True)
df['Weapon Type'].replace('Firearm (unk type)', 'Firearm', inplace=True)
df['Weapon Type'].replace('None/Not Applicable', 'None', inplace=True)
df['Weapon Type'].replace('Brass Knuckles', 'Club, Blackjack, Brass Knuckles', inplace=True)
df['Weapon Type'].replace('Blackjack', 'Club, Blackjack, Brass Knuckles', inplace=True)
df['Weapon Type'].replace('Club', 'Club, Blackjack, Brass Knuckles', inplace=True)
df['Weapon Type'].replace('Shotgun', 'Rifle', inplace=True)

# Officer Year of Birth
df['Officer YOB'] = df['Officer YOB'].astype('int')

# Officer Race
df['Officer Race'].replace('Unknown', np.nan, inplace=True)
df['Officer Race'].replace('Not Specified', np.nan, inplace=True)

# Subject Perceived Race
df['Subject Perceived Race'].replace('-', np.nan, inplace=True)
df['Subject Perceived Race'].replace('Other', np.nan, inplace=True)
df['Subject Perceived Race'].replace('Unknown', np.nan, inplace=True)


# Subject Perceived Gender
df['Subject Perceived Gender'].replace('Unable to Determine', np.nan, inplace=True)
df['Subject Perceived Gender'].replace('-', np.nan, inplace=True)
df['Subject Perceived Gender'].replace('Unknown', np.nan, inplace=True)
df['Subject Perceived Gender'].replace('Male', 'M', inplace=True)
df['Subject Perceived Gender'].replace('Female', 'F', inplace=True)
# Regarding below line: only 4 individuals remained in the non-binary category after dropping rows with na's in relevant fields
# I'm reluctant to remove this category given the potential to investigate one tiny aspect of the societal bias against 
# non-binary folks. However, with a sample size of 4, it is not statistically significant enough to provide meaningful results
df['Subject Perceived Gender'].replace('Gender Diverse (gender non-conforming and/or transgender)', np.nan, inplace=True)


# Reported Date
df['Reported Date'] = df['Reported Date'].astype('datetime64')

# Exclude for now simply because datetime.time shows up as 'object' type during ohe, which caused problems downstream
#if 'datetime.time' not in str(type(df['Reported Time'][0])): # Use if statement to avoid error when re-running
#    df['Reported Time'] = [time.fromisoformat(t) for t in df['Reported Time']]

# Reported Time
df['Reported Hour'] = [int(x[:2]) for x in df['Reported Time']]
    
# Call Type
df['Call Type'].replace('TEXT MESSAGE', '-', inplace=True)
df['Call Type'].replace('SCHEDULED EVENT (RECURRING)', '-', inplace=True)
    
# Frisk Flag
df['Frisk Flag'].replace('-', np.nan, inplace=True)

# Precinct
df['Precinct'].replace('-', 'Unknown', inplace=True)
df['Precinct'].replace('FK ERROR', 'Unknown', inplace=True)
#df['Precinct'].replace('OOJ', 'Unknown', inplace=True)
df['Precinct'].replace('SouthWest', 'Southwest', inplace=True)

# Sector
df['Sector'].replace('99', '-', inplace=True)

# Beat
df['Beat'].replace('S', '-', inplace=True)
df['Beat'].replace('99', '-', inplace=True)


# Categorize "Call Types". Variable used to create dictionary in CSV
call_types = pd.concat([df['Call Type'], df['Initial Call Type'], df['Final Call Type']]).value_counts().keys().values
path = os.path.join('..', 'data', 'categorize call types.csv')
call_type_dictionary = pd.read_csv(path)
call_type_dictionary = dict(zip(call_type_dictionary.iloc[:, 0], call_type_dictionary.iloc[:, 1]))


    
#--- INCOMPLETE!
df['Initial Call Type'] = df['Initial Call Type'].map(call_type_dictionary)
df['Final Call Type'] = df['Final Call Type'].map(call_type_dictionary)
df['Call Type'] = df['Call Type'].map(call_type_dictionary)


In [15]:
# Engineer new features

# Calculate Officer Age
if 'Officer YOB' in df.columns:
    df['Officer Age'] = df['Reported Date'].apply(lambda x: x.year) - df['Officer YOB']
    df['Officer Age'] = df['Officer Age'].apply(lambda x: np.nan if x > 100 else x)
    df.drop('Officer YOB', axis=1, inplace=True)

    
# Determine whether or not weapon was involved
#has_weapon = lambda x: 'No' if x == 'None' else 'Yes'
df['Weapon (Y/N)'] = (df['Weapon Type'] != 'None').astype('int')


# Calculate the number of stops per subject
offense_ct_dict = dict(df['Subject ID'].value_counts())
df['Offense Count'] = df['Subject ID'].map(offense_ct_dict)
# Identify whether a subject has been involved in multiple stops
df['Repeat Offender (Y/N)'] = (df['Offense Count'] > 1).astype('int')
# Drop "Offense Count" for now given the number of missing values
if 'Offense Count' in df:
    df.drop('Offense Count', axis=1, inplace=True)



# Identify whether race of officer is same as race of subject
df['Different Race'] = (df['Officer Race'] != df['Subject Perceived Race']).astype('int')

# Identify whether gender of officer is same as gender of subject
df['Different Gender'] = (df['Officer Gender'] == df['Subject Perceived Gender']).astype('int')

In [16]:
# Rename columns and drop 'na' rows
if 'Arrest Flag' in df.columns:
    df['Arrest (Y/N)'] = (df['Arrest Flag'] == 'Y').astype('int')
    df.drop('Arrest Flag', axis=1, inplace=True)
    
if 'Frisk Flag' in df.columns:
    df['Frisk (Y/N)'] = (df['Frisk Flag'] == 'Y').astype('int')
    df.drop('Frisk Flag', axis=1, inplace=True)
    


In [17]:
# Refine dataset to continous and OHE values

# Separate columns by type
# Identify continuous vs. string type features
str_columns = []
continuous_columns = []
for col in df.columns:
    if type(df.dropna()[col].iloc[0]) == type('abc'):
        str_columns.append(col)
    else:
        continuous_columns.append(col)
# Determine which columns are most valuable to OHE
cols_to_ohe = []
for col in str_columns:
    if len(df[col].value_counts().index) < 30:
        cols_to_ohe.append(col)

df_refined = pd.concat([df[continuous_columns], df[cols_to_ohe]], axis=1)
df_refined.dropna(inplace=True)
df_refined.reset_index(inplace=True, drop=True)

In [18]:
df_refined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40340 entries, 0 to 40339
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Reported Date             40340 non-null  datetime64[ns]
 1   Reported Hour             40340 non-null  int64         
 2   Officer Age               40340 non-null  float64       
 3   Weapon (Y/N)              40340 non-null  int32         
 4   Repeat Offender (Y/N)     40340 non-null  int32         
 5   Different Race            40340 non-null  int32         
 6   Different Gender          40340 non-null  int32         
 7   Arrest (Y/N)              40340 non-null  int32         
 8   Frisk (Y/N)               40340 non-null  int32         
 9   Subject Age Group         40340 non-null  object        
 10  Stop Resolution           40340 non-null  object        
 11  Weapon Type               40340 non-null  object        
 12  Officer Gender    

### Begin encoding and modeling

In [37]:
# Split data
X = df_refined.drop('Arrest (Y/N)', axis=1)
y = df_refined['Arrest (Y/N)']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = RANDOM_STATE)

In [52]:
ohe = OneHotEncoder(drop='first')
string_selector = make_column_selector(dtype_include='object')
string_transformer = make_column_transformer((ohe, string_selector))


logreg = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)

clf = KNeighborsClassifier()

pipe = Pipeline([('string_selector', string_transformer), ('model', logreg)])
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train)

0.9344240621384895

In [53]:
pipe.score(X_test, y_test)

0.9357461576598909

In [21]:
string_transformer

ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(drop='first'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x00000259F3CD2FA0>)])

In [ ]:
string_selector

# notes from study group

Create ModelHistory class like they mentioned during your last one-on-one. 